In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
df=pd.read_csv('/kaggle/input/covid19-symptoms-checker/Cleaned-Data.csv')

In [ ]:
df.head()

In [ ]:
del df['Country']

In [ ]:
df.describe()

In [ ]:
df.drop_duplicates()

# Exploratory data analysis

Heatmap

In [ ]:
figure = plt.figure(figsize=(30,30))
sns.heatmap(df.corr(),annot=True, cmap='coolwarm')
plt.show()

In [ ]:
df.hist(figsize=(30,30))

# Preparing data for analysis

In [ ]:
df.columns

In [ ]:
train_df=df.copy()
        

In [ ]:
#scaling data
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
train_df_scaled=scaler.fit_transform(train_df)

# Elbow method

In [ ]:
inertias = []

for i in range(1,15):
    km=KMeans(n_clusters=i, random_state=2)
    km.fit(train_df_scaled)
    inertias.append(km.inertia_)

In [ ]:


plt.figure(figsize=(10,6))
plt.plot(range(1,15), inertias, color='blue', marker='o', markerfacecolor='red', markersize=10)
plt.title('Inertias vs. number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()



Based on the plot above, I would choose as number of clusters number between 6-8 - I will additionally check Davies Bouldin Score.

In [ ]:
from sklearn.metrics import davies_bouldin_score

In [ ]:
bouldin_score=[]

for i in range(4,15):
    km=KMeans(n_clusters=i, random_state=2)
    labels=km.fit_predict(train_df_scaled)
    bouldin_score.append(davies_bouldin_score(train_df_scaled, labels))



In [ ]:

plt.figure(figsize=(10,6))
plt.plot(range(4,15), bouldin_score, color='blue', marker='o', markerfacecolor='red', markersize=10)
plt.title('Davies Bouldin Score vs. number of clusters')
plt.xlabel('Number of clusters')
plt.ylabel('Davies Bouldin Score')
plt.show()

In Davies Bouldin score we are looking for minimum value, and since minimum value is for 7 clusters, I will use 7 as n_clusters.

# Model - KMeans

In [ ]:
km_model=KMeans(n_clusters=7, random_state=2)
km_model.fit(train_df_scaled)

In [ ]:
labels=km_model.labels_

In [ ]:
corona_df=pd.DataFrame(km_model.cluster_centers_, columns=train_df.columns)

In [ ]:
corona_df

Looks that people from 6th cluster are not affected by covid, unlike other clusters.

# PCA

In [ ]:
covid_pca=PCA(n_components=3)
principal_comp=covid_pca.fit_transform(train_df_scaled)

In [ ]:
principal_comp=pd.DataFrame(principal_comp,columns=['pca1','pca2','pca3'])
principal_comp

In [ ]:
principal_comp1=pd.concat([principal_comp,pd.DataFrame({"Cluster":labels})],axis=1)


In [ ]:
principal_comp1.sample(10)

In [ ]:
#Plotting the 2d-plot.
plt.figure(figsize=(10,10))
ax=sns.scatterplot(x='pca1',y='pca2',hue="Cluster",data=principal_comp1, palette=['red', 'blue', 'yellow', 'black', 'orange', 'green', 'violet'])
plt.show()



In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
sc=ax.scatter(xs=principal_comp1['pca1'],ys=principal_comp1['pca3'],zs=principal_comp1['pca2'],c=principal_comp1['Cluster'],marker='o',cmap="gist_rainbow")
plt.colorbar(sc)
plt.show()